# Simulation

In [1]:
import pandas as pd
import numpy as np
import json
import random
from collections import Counter

pd.options.display.max_rows = 1_000

### Data

In [2]:
proj = pd.read_csv("../data/projections-generated.csv")
proj.head()

,team,name,position,goals,assists,plus_minus,powerplay_points,shots_on_goal,hits,blocks,wins,save_percentage,goals_against_average,saves,shutouts
0,Edm,Connor McDavid,C,30.353,52.693,2.787,23.687,177.727,25.65,22.5,NaN,NaN,NaN,NaN,NaN
1,Col,Nathan MacKinnon,C,28.653,44.883,8.693,21.733,252.167,36.20,20.8,NaN,NaN,NaN,NaN,NaN
2,Edm,Leon Draisaitl,"C,LW",33.487,47.400,-1.777,22.997,164.293,29.80,16.4,NaN,NaN,NaN,NaN,NaN
3,NYR,Artemi Panarin,LW,24.137,45.087,16.927,14.747,159.880,15.20,13.4,NaN,NaN,NaN,NaN,NaN
4,Was,Alex Ovechkin,LW,33.417,20.877,-1.330,14.950,242.383,143.45,22.4,NaN,NaN,NaN,NaN,NaN


In [3]:
df = pd.read_csv("../data/draft-yahoo_league.csv")
df = df.sort_values("pick")
df["position"] = np.where(df["position_yahoo"].isin(["G", "D"]), df["position_yahoo"], "F")
df = df.rename(columns={"pick": 'adp'})
df.head()

,team,age,name,position_yahoo,rollup,vorp,vorn,round,adp,rank,arbitrage,target,position
3,Edm,23.0,Connor McDavid,C,69.6,14.5,8.0,1.0,1.0,4.0,-3.0,True,F
1,Col,25.0,Nathan MacKinnon,C,74.1,19.0,12.5,1.0,2.0,2.0,0.0,True,F
8,Edm,25.0,Leon Draisaitl,"C,LW",65.9,12.2,6.3,1.0,3.0,9.0,-6.0,True,F
12,NYR,29.0,Artemi Panarin,LW,63.3,9.6,3.6,1.0,4.0,13.0,-9.0,True,F
5,Was,35.0,Alex Ovechkin,LW,67.3,13.6,7.7,1.0,5.0,6.0,-1.0,False,F


### Players

In [4]:
players = df[["name", "position", "adp", "vorp"]].to_dict(orient="records")
players[:5]

[{'name': 'Connor McDavid', 'position': 'F', 'adp': 1.0, 'vorp': 14.5},
 {'name': 'Nathan MacKinnon', 'position': 'F', 'adp': 2.0, 'vorp': 19.0},
 {'name': 'Leon Draisaitl', 'position': 'F', 'adp': 3.0, 'vorp': 12.2},
 {'name': 'Artemi Panarin', 'position': 'F', 'adp': 4.0, 'vorp': 9.6},
 {'name': 'Alex Ovechkin', 'position': 'F', 'adp': 5.0, 'vorp': 13.6}]

### Pool Settings

In [5]:
TEAMS = 12
SLOTS = {'F': 6, 'D': 4, 'G': 2}
BENCH = 4
BENCH = 0
PICKS = (sum(SLOTS.values()) + BENCH) * TEAMS 

CATEGORIES = [
    'goals', 
    'assists', 
    'plus_minus', 
    'powerplay_points', 
    'shots_on_goal', 
    'hits', 
    "wins", 
    "save_percentage", 
    "goals_against_average",
    "shutouts"
]

### Draft Order

In [6]:
def snake(low, high, x):
    k = (high - low + 1)
    return k - int(abs(x % (2*k) + low - k - 0.5))

draft_order = [snake(1, 12, x) for x in range(PICKS)]
draft_order[:24]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

### Team Class

In [7]:
class Team:
    
    SLOTS = Counter({'F': 6, 'D': 4, 'G': 2})
    
    def __init__(self, number, pref_slot=0.8, sort_by="adp", sort_reverse=False):
        self.players = []
        self.number = number
        self.pref_slot = pref_slot
        self.sort_by = sort_by
        self.sort_reverse = sort_reverse
      
    
    def __repr__(self): 
        return f"Team({self.number})"

    
    @property
    def slots_to_fill(self):
        """Slots left to fill"""
        slots_to_fill = self.SLOTS.copy()
        if self.players:
            slots_to_fill -= Counter([player["position"] for player in self.players])
        return slots_to_fill
    
    
    def pick(self, available):
        """Simulate pick"""
        player = self._strategy(available)
        self.players.append(player)
        available.remove(player)
        return player
    
    
    def _strategy(self, available):
        """Draft strategy"""
        ranked = sorted(available, key=lambda x: x[self.sort_by], reverse=self.sort_reverse)
        
        # random pick from the top 5 available
        choice_player = random.choices(
            population=ranked[:5], 
            weights=[0.60, 0.20, 0.10, 0.05, 0.05],
            k=1
        )[0]
        
        # chance to ignore slot preference (1-pref_slot)
        if random.uniform(0, 1) > self.pref_slot:
            return choice_player

        # if there are no more slots to fill, pick top
        if not self.slots_to_fill: 
            return choice_player

        # otherwise loop to get the first required slot player
        for player in ranked:
            if self.slots_to_fill.get(player["position"], 0) > 0: 
                return player

In [8]:
class MyTeam:
    def __init__(self, number):
        self.number = number
        self.players = []
        self.round = 1
        
    def pick(self, available):
        print(f"\n=== Round {self.round} ===\n")
        
        slots_to_fill = Counter(SLOTS.copy())
        if self.players:
            slots_to_fill -= Counter([player["position"] for player in self.players])
            print(f"Slots to fill: {slots_to_fill}\n")
        
        if self.players:
            current = [(p["name"], p["position"]) for p in self.players]
            print(current)
        
        ranked = sorted(available, key=lambda x: x["vorp"], reverse=True)[:100]
        
        print("\nAvailable:")
        for i, player in enumerate(ranked):
            print(i, player["name"], player["position"])
            
        pi = int(input("\nPick: "))
        player = ranked[pi]
        
        print(f"\nSelected: {player}")
        self.players.append(player)
        available.remove(player)
        self.round += 1
        return player

### Simulate

In [ ]:
# seed teams
teams = dict()
for number in range(1, TEAMS+1):
    team = Team(number)
    teams[number] = team

# randomize "my" draft pick
me = int(random.uniform(1, TEAMS+1) // 1)

# forced pick 
me = 9
teams[me] = MyTeam(me)
print(f"Team({me})")

# run simulation
available = players.copy()
for i, team in enumerate(draft_order):
    teams[team].pick(available)
    
# "my" picks
teams[me].players

Team(9)

=== Round 1 ===


Available:
0 Nikita Kucherov F
1 Roman Josi D
2 Victor Hedman D
3 Anton Khudobin G
4 Robin Lehner G
5 Dougie Hamilton D
6 John Carlson D
7 Philipp Grubauer G
8 Tuukka Rask G
9 Steven Stamkos F
10 Patrick Kane F
11 Cale Makar D
12 Mikko Rantanen F
13 Connor Hellebuyck G
14 Mika Zibanejad F
15 Jordan Binnington G
16 Brad Marchand F
17 Neal Pionk D
18 Max Pacioretty F
19 Brent Burns D
20 Alex Pietrangelo D
21 Andrei Svechnikov F
22 Kris Letang D
23 J.T. Miller F
24 Carter Hart G
25 Torey Krug D
26 Sidney Crosby F
27 Matthew Tkachuk F
28 Juuse Saros G
29 Evgeni Malkin F
30 Jake Guentzel F
31 Shea Theodore D
32 Blake Wheeler F
33 Mark Stone F
34 Mitchell Marner F
35 Ryan Ellis D
36 Sebastian Aho F
37 Morgan Rielly D
38 Shea Weber D
39 Miro Heiskanen D
40 Zach Werenski D
41 Patrik Laine F
42 Ryan Pulock D
43 Tony DeAngelo D
44 Gabriel Landeskog F
45 Mark Giordano D
46 Teuvo Teravainen F
47 Brady Tkachuk F
48 Johnny Gaudreau F
49 Mikhail Sergachev D
50 Patrice Berge

In [ ]:
df = pd.DataFrame()
for team in teams:
    picks = [player["name"] for player in teams[team].players]
    dt = pd.DataFrame(proj[proj["name"].isin(picks)][CATEGORIES].mean()).T.round(3)
    dt["team"] = team
    df = df.append(dt)
    
df = df.set_index("team")
df.sort_values("assists", ascending=False)

In [ ]:
df.rank(ascending=False).mean(axis=1)

In [ ]:
teams[1].players